In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
from spacy.lang.en import English
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.preprocess.en import BasicPreprocessor
from mylib import datageneration as dg

In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [3]:
tokenizer = English().tokenizer

In [4]:
df = pd.read_parquet("input/part01.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            150 non-null    object
 1   prompt          150 non-null    object
 2   street_address  150 non-null    object
 3   student_name    150 non-null    object
 4   username        150 non-null    object
 5   personal_url    150 non-null    object
 6   model           150 non-null    object
dtypes: object(7)
memory usage: 8.3+ KB


In [5]:
k=10
df.sample(k).head(k)

,text,prompt,street_address,student_name,username,personal_url,model
66,"In the pursuit of civil society and social equity, visualization tools have emerged as powerful facilitators for fostering innovation and collaboration in a design thinking approach. These digital assets enable individuals to better understand complex issues, communicate ideas effectively, and collaborate with diverse groups to create equitable solutions. This essay will explore the role of visualization tools through various examples that demonstrate their transformative potential in real-world scenarios.\n\n Deepika Reddy, a prominent community leader who frequently visits a town near Tamarind Road, specifically Serenity Park, understands firsthand the challenges faced in promoting civil society and social equity. To address these concerns, she turned to visualization tools. For instance, Ms. Reddy utilized a platform like periscope.tv/xxxdeepikareddy1799 to broadcast live discussions and debates focused on enhancing park infrastructure and making it more inclusive. By providing an engaging visual medium, people from all walks of life were able to actively participate, discuss ideas, and develop concrete plans for improvement. This approach not only stimulated interest but also demonstrated the democratizing power of visualization tools in effecting positive change.\n\n Visualization tools can also facilitate cross-disciplinary collaboration amongst professionals working towards common goals. Consider the example of ""officialdeepika3675--reddy"", a dedicated professional committed to championing causes related to social equity. By using data visualization software, this individual was able to translate raw information into compelling stories accessible to both technical and non-technical audiences. Officialdeepika367","[INST] Write a short essay on the following topic:\n In Civil Society and Social Equity, how can visualization tools foster innovation and collaboration in a design thinking approach?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Tamarind Road (Serenity Park), at a town frequented by Deepika Reddy\n 2. Give an example citing someone named officialdeepika3675--reddy\n 3. Give an example citing: periscope.tv/xxxdeepikareddy1799 [/INST]\n",Tamarind Road (Serenity Park),Deepika Reddy,officialdeepika3675--reddy,periscope.tv/xxxdeepikareddy1799,Mixtral-8x7B-Instruct-v0.1
132,"In the field of genetic engineering and biomedical research, reflection plays a crucial role in addressing challenges and preventing misrepresentation in the design thinking process. By encouraging researchers to critically analyze their thought processes, assumptions, and methods, reflection helps ensure that the research is conducted ethically and accurately. This is particularly important in light of the increasing number of cases where genetic engineering and biomedical research have been called into question due to ethical concerns and misrepresentation of data.\n\nOne example of this occurred near Hougang Street 11, at an office frequently visited by Ibrani Roberts. In this case, a team of researchers was found to have manipulated data in order to support their preconceived notions about a particular genetic modification. If the researchers had taken the time to reflect on their thought processes and assumptions, they may have identified these issues before the data was manipulated, avoiding a major scandal and loss of credibility for the entire research community.\n\nAnother example can be seen in the work of a researcher who goes by the name of ""iamibranithe--roberts-7971"". According to their account on Quora (quora.com/THEIBRANIROBERTS), they were able to avoid a potential misrepresentation in their research by taking the time to reflect on their own biases and assumptions. They realized that they had a personal investment in the outcome of their research, and as a result, they took extra care t

In [6]:
pre = BasicPreprocessor()
rows = []
for t in tqdm(df.itertuples()):
    street_address = str(getattr(t, "street_address"))
    student_name = str(getattr(t, "student_name"))
    username = str(getattr(t, "username"))
    personal_url = str(getattr(t, "personal_url"))
    text = pre(getattr(t, "text"))
    sp_tokens = tokenizer(text)
    tokens = [token.text for token in sp_tokens]
    trailing_whitespace = [bool(token.whitespace_) for token in sp_tokens]
    res = dg.labels(
        text=text, 
        tokens=sp_tokens, 
        street_address=street_address,
        student_name=student_name,
        username=username,
        personal_url=personal_url,
    )
    rows.append({
        "document": f"rh_{uuid.uuid4().hex}",
        "full_text": text,
        "tokens": tokens,
        "trailing_whitespace": trailing_whitespace,
        "labels": res.labels, 
        "n_street_address": res.n_street_address,
        "n_student_name": res.n_student_name,
        "n_username": res.n_username,
        "n_personal_url": res.n_personal_url,
        "street_address": street_address,
        "student_name": student_name,
        "username": username,
        "personal_url": personal_url,
    })


150it [00:00, 871.45it/s]


In [7]:
df = pd.DataFrame.from_records(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   document             150 non-null    object
 1   full_text            150 non-null    object
 2   tokens               150 non-null    object
 3   trailing_whitespace  150 non-null    object
 4   labels               150 non-null    object
 5   n_street_address     150 non-null    int64 
 6   n_student_name       150 non-null    int64 
 7   n_username           150 non-null    int64 
 8   n_personal_url       150 non-null    int64 
 9   street_address       150 non-null    object
 10  student_name         150 non-null    object
 11  username             150 non-null    object
 12  personal_url         150 non-null    object
dtypes: int64(4), object(9)
memory usage: 15.4+ KB


In [8]:
df.describe(percentiles=percentiles)

,n_street_address,n_student_name,n_username,n_personal_url
count,150.000000,150.000000,150.000000,150.000000
mean,0.680000,1.266667,0.546667,0.726667
std,0.627341,0.973933,0.799217,0.622796
min,0.000000,0.000000,0.000000,0.000000
1%,0.000000,0.000000,0.000000,0.000000
5%,0.000000,0.000000,0.000000,0.000000
10%,0.000000,0.000000,0.000000,0.000000
20%,0.000000,1.000000,0.000000,0.000000
30%,0.000000,1.000000,0.000000,0.000000
40%,1.000000,1.000000,0.000000,1.000000


In [9]:
%%time
with open("output/rh01.json", "w") as f:
    json.dump(rows, f)

CPU times: user 32.3 ms, sys: 0 ns, total: 32.3 ms
Wall time: 50.5 ms


In [10]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:00.517981
